<a href="https://colab.research.google.com/github/MikeKertesz/Airbnb-Data-Analysis-Project/blob/main/Airbnb_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

# Example with listings.csv
listings_clean = pd.read_csv("clean_listings.csv", low_memory=False)
calendar_clean = pd.read_csv("clean_calendar.csv", low_memory=False)
reviews = pd.read_csv("reviews.csv", low_memory=False)
neighbourhoods = pd.read_csv("neighbourhoods.csv")

In [4]:
import sqlite3

conn = sqlite3.connect("airbnb.db")

listings_clean.to_sql("listings_clean", conn, if_exists="replace", index=False)
calendar_clean.to_sql("calendar_clean", conn, if_exists="replace", index=False)
reviews.to_sql("reviews", conn, if_exists="replace", index=False)
neighbourhoods.to_sql("neighbourhoods", conn, if_exists="replace", index=False)
conn.close()

clean_listings - id,name,host_id,host_name,neighbourhood, neighbourhood_group, latitude, longitude, room_type, price, minimum_nights, number_of_reviews, last_review, reviews_per_month, calculated_host_listings_count, availability_365, number_of_reviews_ltm

clean_calendar - listing_id, date, available, minimum_nights, maximum_nights

reviews - listing_id, date

neighbourhood - neighbourhood, neighbourhood_group


In [5]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("airbnb.db")

query = """
SELECT neighbourhood_group, room_type, ROUND(AVG(price),2) AS avg_price
FROM listings_clean
GROUP BY neighbourhood_group, room_type
ORDER BY avg_price DESC
LIMIT 10;
"""

pd.read_sql(query, conn)

,neighbourhood_group,room_type,avg_price
0,Sant Martí,Hotel room,337.67
1,Eixample,Entire home/apt,328.26
2,Ciutat Vella,Hotel room,277.00
3,Gràcia,Hotel room,266.30
4,Eixample,Hotel room,258.31
5,Sants-Montjuïc,Entire home/apt,257.90
6,Sant Martí,Entire home/apt,250.42
7,Horta-Guinardó,Entire home/apt,238.01
8,Gràcia,Entire home/apt,237.19
9,Sant Andreu,Entire home/apt,215.09


In [23]:
conn = sqlite3.connect("airbnb.db")

query = """
SELECT neighbourhood, neighbourhood_group, room_type, ROUND(AVG(price),2) AS avg_price
FROM listings_clean
GROUP BY neighbourhood_group, neighbourhood
ORDER BY avg_price DESC
LIMIT 10;
"""
pd.read_sql(query, conn)

,neighbourhood,neighbourhood_group,room_type,avg_price
0,el Coll,Gràcia,Entire home/apt,465.31
1,la Dreta de l'Eixample,Eixample,Entire home/apt,316.84
2,Diagonal Mar i el Front Marítim del Poblenou,Sant Martí,Entire home/apt,314.15
3,Sants - Badal,Sants-Montjuïc,Entire home/apt,313.94
4,la Nova Esquerra de l'Eixample,Eixample,Entire home/apt,294.04
5,Sant Antoni,Eixample,Entire home/apt,269.95
6,la Vila Olímpica del Poblenou,Sant Martí,Entire home/apt,267.11
7,Navas,Sant Andreu,Entire home/apt,244.57
8,el Besòs i el Maresme,Sant Martí,Entire home/apt,240.36
9,Hostafrancs,Sants-Montjuïc,Entire home/apt,230.25


In [11]:

conn = sqlite3.connect("airbnb.db")

query = """
SELECT  name, neighbourhood_group,room_type, price
FROM listings_clean
ORDER BY price DESC
LIMIT 10;"""
pd.read_sql(query, conn)

,name,neighbourhood_group,room_type,price
0,Apartamento cerca de la playa,Sant Martí,Entire home/apt,10000.0
1,Amazing loft-style apartment in Las Ramblas,Ciutat Vella,Entire home/apt,9999.0
2,Estudio cerca de Park Güell,Gràcia,Entire home/apt,9714.0
3,Modern Flat next to Bogatell beach,Sant Martí,Entire home/apt,9286.0
4,Penthouse Girasol,Horta-Guinardó,Entire home/apt,9143.0
5,Pierre & Vacances - Studio 3 personnes,Sants-Montjuïc,Entire home/apt,9059.0
6,Pierre&Vacances - Appart 4 personnes - 1 chambre,Sants-Montjuïc,Entire home/apt,9059.0
7,Pierre&Vacances - Appart 5 personnes - 1 chambre,Sants-Montjuïc,Entire home/apt,9059.0
8,Pierre & Vacances - Studio 4 Personnes,Sants-Montjuïc,Entire home/apt,9059.0
9,Navas 2,Sant Andreu,Entire home/apt,9007.0


In [19]:
conn = sqlite3.connect("airbnb.db")

query = """
SELECT minimum_nights, count(minimum_nights), ROUND(AVG(price),2) AS avg_price
FROM listings_clean
GROUP BY minimum_nights
ORDER BY minimum_nights
LIMIT 10;
"""

pd.read_sql(query, conn)

,minimum_nights,count(minimum_nights),avg_price
0,1,4479,352.27
1,2,1540,239.48
2,3,1609,241.17
3,4,480,252.15
4,5,244,239.41
5,6,70,258.33
6,7,93,185.61
7,8,3,115.00
8,9,2,147.00
9,10,13,97.69


In [21]:
conn = sqlite3.connect("airbnb.db")

query = """
SELECT room_type, avg(minimum_nights), ROUND(AVG(price),2) AS avg_price
FROM listings_clean
GROUP BY room_type
ORDER BY avg_price
LIMIT 10;
"""
pd.read_sql(query, conn)

,room_type,avg(minimum_nights),avg_price
0,Private room,19.872447,94.22
1,Shared room,6.055556,124.46
2,Hotel room,1.471698,260.87
3,Entire home/apt,14.800265,262.84


In [29]:
conn = sqlite3.connect("airbnb.db")

query = """
SELECT l.neighbourhood, avg(l.price),
       ROUND(AVG(cal.maximum_nights),2) AS avg_max,
       COUNT(cal. listing_id) AS num_reviews
FROM listings_clean l
LEFT JOIN calendar_clean cal ON l.id = cal.listing_id
GROUP BY l.neighbourhood
ORDER BY num_reviews DESC;
"""
pd.read_sql(query, conn)

,neighbourhood,avg(l.price),avg_max,num_reviews
0,la Dreta de l'Eixample,288.163215,3561913.96,220096
1,el Raval,141.101072,529.18,123735
2,la Sagrada Família,210.863531,611.94,121910
3,la Vila de Gràcia,201.855180,634.91,114975
4,"Sant Pere, Santa Caterina i la Ribera",161.166346,528.66,105850
...,...,...,...,...
65,Canyelles,201.000000,1125.00,365
66,Can Peguera,33.000000,90.00,365
67,Torre Baró,136.500000,NaN,0
68,Sant Genís dels Agudells,97.857143,NaN,0
